In [1]:
import pandas as pd
import random
import math
import numpy as np

In [2]:
#data = pd.read_csv('./wordle_data/possible_words.txt', header = None)
data = pd.read_csv('./wordle_data/allowed_words.txt', header = None)
data.columns = ['word']
data.head()

,word
0,aahed
1,aalii
2,aargh
3,aarti
4,abaca


In [3]:
def compare_word(w_target,w_guess):
    out = []
    for i in range(5):
        temp = 0
        if w_guess[i] in w_target:
            temp = 1
            if w_guess[i] == w_target[i]:
                temp = 2
        else:
            temp  = 0
        out.append(temp)
    return out

In [4]:
compare_word('chick', 'crane')

[2, 0, 0, 0, 0]

In [5]:
def wordle_game(df):
    st = str(input('Start worlde game? y or n? '))
    if st == 'y':
        target = random.choice(df.word)
        state = 0
        temp_sum = 0
        data_his = []
        guess_his = []
        print('=======')
        while (state <6) and (temp_sum<10):
            method = 1
            guess = choose_word(data_his, guess_his,df,method)
            while guess not in list(df.word):
                guess = str(input('WORD not in the list, guess again: '))
            guess_his.append(guess)
            out_compare = compare_word(target, guess)
            data_his.append(out_compare)
            print(', '.join(guess))
            print(', '.join([str(x) for x in out_compare]))
#            print([char for char in guess])
#            print([str(x) for x in out_compare])
            print()
            state = state + 1
            temp_sum = sum(out_compare)
        if temp_sum == 10:
            print('WINNN')
        else:
            print('LOSE HIU HIU, target word is "{}"'.format(target))
        return None
    
def choose_word(data_his, guess_his, df,method):
    if method == 0:
        gss = str(input('Guess b*tch: '))
        return gss
    if method == 0.5:
        all_word = find_all_word(data_his, guess_his, df)
        if len(data_his) > 0:
            if len(all_word) >= 5:
                print('Posible word: ' + ', '.join(all_word[:4]))
            else:
                print('Posible word: ' + ', '.join(all_word))
        gss = str(input('Guess b*tch: '))
        return gss
    if method == 1:
        if len(data_his) > 0:
            all_word = find_all_word(data_his, guess_his, df)
            if len(all_word) >= 5:
                print('Posible word: ' + ', '.join(all_word[:4]))
            else:
                print('Posible word: ' + ', '.join(all_word))
            max_en, en_score = calculate_totel_entropy(all_word, df)
            en_score = [str(np.round(x,3)) for x in en_score]
            win_able = len(data_his) + len(all_word)
            if win_able > 6:
                print('Best guess: ' + ', '.join(max_en))
                print('Score guess: ' + ', '.join(en_score))
        gss = str(input('Guess b*tch: '))
        return gss
    if method == 2:
        if len(data_his) > 0:
            all_word = find_all_word(data_his, guess_his, df)
            win_able = len(data_his) + len(all_word)
            if win_able > 6:
                max_en, en_score = calculate_totel_entropy_auto(all_word, df)
                return max_en
            else:
                return all_word[0]
        else:
            return 'crane'
        

In [13]:
wordle_game(data)

Start worlde game? y or n?  y


Guess b*tch:  crane


c, r, a, n, e
0, 0, 1, 0, 1

Posible word: aahed, abbed, abbes, abbey
Best guess: dalts, malts, halts, delts, talas, tales, talks, palms, melts, balds
Score guess: 5.127, 5.087, 4.906, 4.891, 4.869, 4.857, 4.853, 4.846, 4.83, 4.822


Guess b*tch:  dalts


d, a, l, t, s
1, 1, 0, 0, 0

Posible word: abbed, added, adeem, adieu
Best guess: imbed, humid, aphid, hided, embed, muxed, bemud, bided, bimah, bedim
Score guess: 3.584, 3.541, 3.484, 3.484, 3.484, 3.422, 3.422, 3.384, 3.346, 3.346


...

i, m, b, e, d
0, 0, 1, 1, 2

Posible word: bedad, begad


Guess b*tch:  bedad


b, e, d, a, d
2, 2, 1, 2, 2

Posible word: begad


Guess b*tch:  begad


b, e, g, a, d
2, 2, 2, 2, 2

WINNN


In [7]:
def find_all_word(data_his, guess_his,df):
    lsts  = list(df.word)
    out = []
    for lst in lsts:
        if check_meet_his(lst,data_his, guess_his):
            out.append(lst)
    return out
        
def check_meet_his(guess, data_his, guess_his):
    for i in range(len(guess_his)):
        temp = compare_word(guess, guess_his[i])
        tempp = data_his[i]
        for j in range(5):
            if temp[j] != tempp[j]:
                return False
    return True

In [8]:
dt_his = [[0, 0, 0, 0, 1],[0, 1, 0, 0, 0],[0, 2, 1, 0, 0]]
gss_his = ['crane','fight','melon']
print(check_meet_his('kevil', dt_his, gss_his))
print(find_all_word(dt_his, gss_his,data))

True
['bezil', 'deils', 'devil', 'kevil', 'lepid', 'levis', 'lewis', 'lexis', 'seils', 'veils', 'veily', 'vexil', 'weils']


In [9]:
def calculate_entropy(guess, all_pool):
    lst_score = []
    for w in all_pool:
        out_compare = compare_word(w, guess)
        temp = 0
        for i in range(5):
            temp = 3 * temp + out_compare[i]
        lst_score.append(temp)
    set_score = set(lst_score)
    fre = {x:lst_score.count(x) for x in set_score}
    temp = 0
    for js in set_score:
        temp = temp + math.log2(len(all_pool)/fre[js])*fre[js]/len(all_pool)
    return temp

def calculate_totel_entropy(all_pool, df):
    list1 = list(df.word)
    list2 = []
    for il in list1:
        score = calculate_entropy(il, all_pool)
        list2.append(score)
    list2, list1 = zip(*sorted(zip(list2, list1), reverse = True))
    return list1[:10], list2[:10]

def calculate_totel_entropy_auto(all_pool, df):
    list1 = list(df.word)
    list2 = []
    for il in list1:
        score = calculate_entropy(il, all_pool)
        list2.append(score)
    idx = list2.index(max(list2))
    return list1[idx], list2[idx]

# Automation

In [16]:
def wordle_game_auto(df,idd,method):
    print(idd)
    target = list(df.word)[idd]
    state = 0
    temp_sum = 0
    data_his = []
    guess_his = []
#    print('=======')
    while (state <6) and (temp_sum<10):
        guess = choose_word(data_his, guess_his,df,method)
        guess_his.append(guess)
        out_compare = compare_word(target, guess)
        data_his.append(out_compare)
#        print(', '.join(guess))
#        print(', '.join([str(x) for x in out_compare]))
#            print([char for char in guess])
#            print([str(x) for x in out_compare])
        state = state + 1
        temp_sum = sum(out_compare)
    if temp_sum == 10:
        return 1, len(guess_his)
    else:
        return 0, 7

In [12]:
len(data)

12972

In [20]:
test = [1,2,3]
np.save('test.npy', test)